In [ ]:
import xarray as xr
from glob import glob
from os import path
import os
import numpy as np


# Create list of CCMP filenames

In [ ]:
dir_pattern = 'F:/data/sat_data/ccmp/v02.0/'
dir_pattern_zarr = 'F:/data/sat_data/ccmp/zarr2/'
pattern = 'F:/data/sat_data/ccmp/v02.0/*/*/*_V02.0_L3.0_RSS.nc'

In [ ]:
files = [x for x in glob(pattern)]
print(files[0])
print(files[-1])

In [ ]:
%%time
#list files
files = [x for x in glob(pattern)]
#open dataset
ds=xr.open_mfdataset(files,combine='nested',concat_dim='time',decode_cf=False,mask_and_scale=False)
# if you don't need all the variables, drop extra ones here
# ds = ds.drop({'nobs'})
ds.close()

In [ ]:
#remove any duplicates -- CCMP has both NRT and RT data so some files are duplicates -- 
# this returns 1st occurance check this is what you want
_, index = np.unique(ds['time'], return_index=True)
ds = ds.isel(time=index)

# Set Chunk Size Here

- you want chunks of about 100mb

In [ ]:
#rechunck data      #cci SUGGESTION
itime_chunk = 2000   #200
ilat_chunk = 157    #300
ilon_chunk = 180    #600

ds = ds.chunk({'time':itime_chunk,'latitude':ilat_chunk,'longitude':ilon_chunk})

ds

In [ ]:
%%time
#only need to run once to write data
ds.to_zarr(dir_pattern_zarr, consolidated=True)

# Test reading zarr file and making some plots

In [ ]:
%%time
dir_pattern_zarr = 'F:/data/sat_data/ccmp/zarr2/'
ds2= xr.open_zarr(dir_pattern_zarr)
ts = ds2.uwnd.sel(latitude=slice(-60,0),longitude=slice(70,180)).mean({'latitude','longitude'}).plot()


In [ ]:
%%time
dir_pattern_zarr = 'F:/data/sat_data/ccmp/zarr/'
ds2= xr.open_zarr(dir_pattern_zarr)
ts = ds2.uwnd.sel(latitude=slice(-60,0),longitude=slice(70,180)).mean({'latitude','longitude'}).plot()


# upload to S3

In [ ]:
#! pip install awscli 
! aws s3 sync F:\data\sat_data\ccmp\zarr\ s3://cmc-sst/zarr

# update in place and then on AWS

In [ ]:
import xarray as xr
from glob import glob
from os import path
import os
import numpy as np

dir_pattern = 'F:/data/sat_data/ccmp/v02.0/'
dir_pattern_zarr = 'F:/data/sat_data/ccmp/zarr2/'
pattern = 'F:/data/sat_data/ccmp/v02.0/*/*/*_V02.0_L3.0_RSS.nc'

files = glob(pattern)
print(files[0])
print(files[-1])


In [ ]:
ds_old = xr.open_zarr(dir_pattern_zarr, consolidated=True,decode_cf=False)
ds_old.close()
lasttime=ds_old.time[-1] + 1#+np.timedelta64(1, 'D')
print(lasttime.data)
ds_old

In [ ]:
ds_old.time[-4:].data

In [ ]:
ds=xr.open_mfdataset(files[-60:],combine='nested',concat_dim='time',decode_cf=False,mask_and_scale=False)
ds.close()
ds_all = ds
#remove any duplicates
_, index = np.unique(ds_all['time'], return_index=True)
ds_all=ds_all.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
itime_chunk = 2000   #200
ilat_chunk = 157    #300
ilon_chunk = 180    #600
ds_all = ds_all.chunk({'time':itime_chunk,'latitude':ilat_chunk,'longitude':ilon_chunk})
ds_new = ds_all.sel(time=slice(lasttime,9999999))
ds_new = xr.decode_cf(ds_new)


In [ ]:
print(ds_old.time[-4:].data)
print(ds_new.time[:4].data)

In [ ]:
ds_new.to_netcdf('f:/tem/ccmp.nc')
ds_new.to_zarr(dir_pattern_zarr, mode='a',append_dim='time', consolidated=True) #


In [ ]:
ds_old = xr.open_zarr(dir_pattern_zarr, consolidated=True)
ds_old.close()
ds_old

In [ ]:
ds_old.uwnd[-1,:,:].plot()